In [1]:
from bs4 import BeautifulSoup, element
import os
import re
import sys

In [2]:
tar_bible = BeautifulSoup(open('diccionario.html'))

In [3]:
styles_to_names = {}
names_to_styles = {}
for line in tar_bible.head.style.contents[0].split('\n'):
    if line.strip().startswith('.'):
        name, style = line.split('{')
        name, style = name.strip(' .'), style.strip(' .}')
        names_to_styles[name] = style
        aliases = styles_to_names.get(style, [])
        aliases.append(name)
        styles_to_names[style] = aliases
        
styles_to_names
        

{'font-size:24pt; font-family:Liberation Serif; min-height:0.422cm;min-width:0.422cm;padding-top:0cm; padding-bottom:0cm; padding-left:0cm; padding-right:0cm;': ['gr1'],
 'font-size:24pt; font-family:Liberation Serif; min-height:0.321cm;min-width:0.321cm;padding-top:0cm; padding-bottom:0cm; padding-left:0cm; padding-right:0cm;': ['gr2'],
 'font-size:24pt; font-family:Liberation Serif; min-height:0.355cm;min-width:0.355cm;padding-top:0cm; padding-bottom:0cm; padding-left:0cm; padding-right:0cm;': ['gr3'],
 'font-size:24pt; font-family:Liberation Serif; min-height:0.566cm;min-width:0.566cm;padding-top:0cm; padding-bottom:0cm; padding-left:0cm; padding-right:0cm;': ['gr4'],
 'font-size:24pt; font-family:Liberation Serif; min-height:0.702cm;min-width:0.702cm;padding-top:0cm; padding-bottom:0cm; padding-left:0cm; padding-right:0cm;': ['gr5'],
 'text-align:left ! important; writing-mode:lr-tb;': ['P1'],
 'font-size:12pt;': ['P2'],
 'font-size:9.10000038146973pt;': ['P3'],
 'font-size:10.1000

In [5]:

prev_name = ''
chapter = ''
verse = ''

with open("train-diccionario.txt", "w") as text:
    
    for page in tar_bible.body.find_all(class_="dp2"):
        divs = page.find_all('div')

        first, last = 0, len(divs)
        for d in divs[first:last]:
            if d['style']:
                height, width, padding = d['style'].split(';', 2)
                #print(width.split(':')[1])
                step = ''

                for span in d.find_all('span'):
                    style = names_to_styles.get(span['class'][0])
                    name = styles_to_names.get(style)[0]
                    #print(f'[{name}] ', end='')

                    if name == 'T6':
                        if prev_name == 'T7' and note == 'Véase':
                            note = ''
                            # print(f'[{span.text.strip()}]')
                            continue
                        if prev_name != 'T6':
                            print(file=text)
                            
                        print(span.text.strip(' ][().\xa0¹²³'), end=" ", file=text)
                        if step == "INFL":
                            prefix = {
                                "fut.": "va a",
                                "pret.": "acaba de",
                                "sing.": "uno",
                                "pl.": "varios",
                                "pp.": "que"
                            }.get(infl, f"({infl})")
                            print(f"\t{prefix} {prev_def}", file=text)
                    elif name == 'T3':
                        note = span.text.strip()
                        if not note and prev_name == 'T6':
                            # probably at a new headword
                            step = "HEAD"
                            headword = prev_span.text.strip()
                            print(end=' ', file=text)
                        elif note == ":":
                            # probably inflecting the headword
                            step = "INFL"
                        elif note == ";":
                            # probably starting a new inflection
                            step = "INFL"
                        elif step == "DEF" and span.text.strip():
                            step = "EXE"
                            m = re.search(f'[0-9]\. ', span.text)
                            if m:
                                pieces = [p.strip() for p in re.split(r'[0-9]\. ', span.text) if p.strip()]
                                if pieces:
                                    print('\t' + pieces[0], file=text)
                                    for p in pieces[1:]:
                                        print(f'\n{headword}\t{p}', file=text)
                            elif span.text.strip(' ][().\xa0'):
                                if prev_name != 'T3':
                                    print("\t", end="", file=text)
                                print(span.text.strip(' ][().\xa0'), end=" ", file=text)
                        else:
                            # probably at a definition
                            m = re.search(r'[0-9]\. ', span.text)
                            if m:
                                pieces = [p.strip() for p in re.split(r'[0-9]\. ', span.text) if p.strip(' ][().\xa0')]
                                if pieces:
                                    print('\t' + pieces[0], file=text)
                                    for p in pieces[1:]:
                                        print(f'\n{headword}\t{p}', file=text)
                            elif span.text.strip(' ][().'):
                                if step == "HEAD":
                                    prev_def = span.text.strip()
                                if prev_name != 'T3':
                                    print("\t", end="", file=text)
                                print(span.text.strip(' ][().'), end=" ", file=text)
                            step = "DEF"

                    elif name == 'T7':
                        note = span.text.strip()
                        # print(f"[{note}]", end="\t")
                        infl = note
#                     elif name == 'T14':
#                         verse = span.text.strip()
# #                     elif name in ('T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T8', 'T9', 'T11', 'T15'):
# #                         continue
#                     else:
#                         if name != prev_name:
#                             print(f'\n{chapter}:{verse}', end='\t')
#                             #text.write(f'\n{chapter}:{verse}\t')
                    #print(span.text.strip(), end=' ')
                    #text.write(span.text.strip() + ' ')
        
                    prev_name = name
                    prev_span = span

In [128]:
es_bible_dir = "/Users/lucienc/Downloads/spavbl_html"

index = BeautifulSoup(open(os.path.join(es_bible_dir, "index.htm")))
books = index.find("ul")
with open("train-biblia.es", "w") as text:
    for b in books.find_all("a", class_="nn"):
        this_chapter = BeautifulSoup(open(os.path.join(es_bible_dir, b["href"])))
        while this_chapter:
            chapter = this_chapter.find(class_="chapterlabel").text
            for p in this_chapter.find_all(class_="p"):
                for c in p.children:
                    if isinstance(c, element.Tag):
                        if "verse" in c["class"]:
                            print(f"\n{chapter}:{c.text}" + '\t', end='')
                            text.write(f"\n{chapter}:{c.text}" + '\t')
                        elif "notemark" in c["class"]:
                            continue
                    else:
                        print(c, end='')
                        text.write(c)
            tnav = this_chapter.find_all(class_="tnav")[0]
            next_chapter = tnav.find_all("li")[-1].a["href"]
            # print(next_chapter)
            if next_chapter.startswith(b["href"][:3]):
                this_chapter = BeautifulSoup(open(os.path.join(es_bible_dir, next_chapter)))
            else:
                this_chapter = None

 
 1:1 	Este libro es el registro de Jesús el Mesías, Hijo de David, Hijo de Abraham, comenzando con el linaje de su familia:  
 1:2 	Abraham fue el padre de Isaac; e Isaac el padre de Jacob; y Jacob el padre de Judá y de sus hermanos;  
 1:3 	y Judá fue el Padre de Fares y Zarah (su madre fue Tamar); y Fares fue el padre de Esrom; y Esrom el padre de Ram;  
 1:4 	y Ram fue el padre de Aminadab; y Aminadab el padre de Nasón; y Nasón el padre de Salmón;  
 1:5 	y Salmón el padre de Booz (su madre fue Rahab); y Booz el padre de Obed (su madre fue Rut); y Obed el padre de Isaí;  
 1:6 	e Isaí el padre del Rey David. David fue el padre de Salomón (su madre había sido la esposa de Urías);  
 1:7 	y Salomón el padre de Roboam; y Roboam el padre de Abías; y Abías el padre de Asa;  
 1:8 	y Asa fue el padre de Josafat; y Josafat el padre de Yoram; y Yoram el padre de Uzías;  
 1:9 	y Uzías fue el padre de Jotam; y Jotam el padre de Acaz; y Acaz el padre de Ezequías;  
 1:10 	y Ezequías el padr

 14:31 	De inmediato Jesús se extendió y lo tomó, y le dijo: “Tienes tan poca confianza en mi. ¿Por qué dudaste?”  
 14:32 	Y cuando entraron a la barca, el viento dejó de soplar.  
 14:33 	Y los que estaban en la barca lo adoraban, diciendo: “¡Realmente eres el Hijo de Dios!”  
 14:34 	Después de cruzar el lago, llegaron a Genesaret.  
 14:35 	Cuando la gente de allí se dio cuenta de que era Jesús, lo hicieron saber a todos en la región. Entonces trajeron ante Jesús a todos los que estaban enfermos,  
 14:36 	y le imploraban que dejara que los enfermos tan solo tocasen su manto. Todos los que lo tocaban eran sanados.  
 15:1 	Entonces algunos Fariseos y maestros religiosos de Jerusalén vinieron a Jesús y le preguntaron:  
 15:2 	“¿Por qué tus discípulos quebrantan la tradición de nuestros antepasados al no lavar sus manos antes de comer?”  
 15:3 	“¿Por qué ustedes quebrantan el mandamiento por causa de su tradición?” respondió Jesús.  
 15:4 	“Pues Dios dijo: ‘Honra a tu padre y a tu

 25:17 	De la misma manera, el que tenía dos talentos obtuvo otros dos.  
 25:18 	Pero el que recibió un talento se fue y cavó un hoyo y escondió allí el dinero de su amo.  
 25:19 	Mucho tiempo después, el amo de estos siervos regresó y se dispuso a ajustar cuentas con ellos.  
 25:20 	El que recibió cinco talentos vino y presentó otros cinco talentos. ‘Mi señor,’ le dijo, ‘me diste cinco talentos. Mira, obtuve ganancia de cinco talentos más.’  
 25:21 	Su amo le dijo: ‘has hecho bien, eres un siervo bueno y fiel. Has demostrado que eres fiel en cosas pequeñas, así que ahora te colocaré a cargo de muchas cosas. ¡Alégrate porque estoy muy complacido de ti!  
 25:22 	El siervo que recibió dos talentos también vino. ‘Mi señor,’ le dijo, ‘me entregaste dos talentos. Mira, he obtenido ganancia de dos talentos más.’  
 25:23 	Su amo le dijo: ‘has hecho bien, eres un siervo bueno y fiel. Has demostrado que eres fiel en cosas pequeñas, así que ahora te pondré a cargo de muchas cosas. ¡Alégrat

 6:25 	La joven se apresuró para regresar y le dijo al rey: “Quiero que me des ahora la cabeza de Juan el Bautista en un plato.”  
 6:26 	El rey estaba muy descontento por esto, pero como había hecho juramento frente a sus invitados, no quiso negarse a lo que ella pidió.  
 6:27 	Así que inmediatamente envió a un verdugo para que le trajera la cabeza de Juan. Después de decapitarlo en la prisión,  
 6:28 	el verdugo trajo la cabeza de Juan en un plato y se lo entregó a la niña, y ella se lo entregó a su madre.  
 6:29 	Cuando los discípulos de Jesús oyeron lo que había ocurrido, vinieron y tomaron su cuerpo y lo colocaron en una tumba.  
 6:30 	Los apóstoles regresaron y se reunieron alrededor de Jesús. Le contaron todo lo que habían hecho y lo que habían enseñado.  
 6:31 	“Vengan conmigo, solo ustedes. Iremos a un lugar tranquilo, y descansaremos un poco,” les dijo Jesús, pues surgían tantas cosas por todas partes que ni siquiera tenían tiempo de comer.  
 6:32 	Así que se fueron en 

 14:58 	“Lo oímos decir: ‘Destruiré este templo que han construido manos humanas, y en tres días lo volveré a construir sin usar las manos.’ ”  
 14:59 	Pero aun así sus testimonios no concordaban.  
 14:60 	Entonces el sumo sacerdote se puso en pie frente al concilio, y le preguntó a Jesús: “¿No tienes nada que decir en respuesta a las acusaciones que se han hecho en tu contra?”  
 14:61 	Pero Jesús permanecía en silencio y no respondía. Así que el sumo sacerdote le preguntó de nuevo: “¿eres el Mesías, el Hijo del Dios bendito?”  
 14:62 	“Lo soy,” respondió Jesús, “y ustedes verán al Hijo del hombre sentado a la diestra del Todopoderoso, y viniendo en las nubes del cielo.”  
 14:63 	Entonces el sumo sacerdote rasgó sus vestidos y preguntó: “¿Por qué necesitamos más testigos?  
 14:64 	¡Ustedes mismos han escuchado la blasfemia! ¿Qué tienen para decir?” Y todos lo hallaron culpable y lo condenaron a muerte.  
 14:65 	Entonces algunos de ellos comenzaron a escupirlo. Le vendaron los oj

...

 5:33 	Sin embargo, cada esposo debe amar a su propia esposa como a sí mismo, y la esposa debe respetar a su esposo.  
 6:1 	Hijos, hagan lo que sus padres les dicen, porque esto es lo correcto.  
 6:2 	“Honra a tu padre y a tu madre.” Este es el primer mandamiento que tiene una promesa unida:  
 6:3 	“para que te vaya bien y tengas larga vida en la tierra.” 
 6:4 	Padres, no enojen a sus hijos, sino cuiden de ellos, disciplinándolos e instruyéndolos acerca de Dios.  
 6:5 	Siervos, obedezcan a sus amos en la tierra, con el debido respeto y admiración, haciendo las cosas con sinceridad, como si sirvieran a Cristo.  
 6:6 	No trabajen simplemente cuando los ven o para recibir aprobación, sino trabajen como siervos de Cristo, haciendo con honestidad la voluntad de Dios,  
 6:7 	sirviendo con alegría, como si lo hicieran para el Señor y no para la gente.  
 6:8 	Ustedes saben que todo el que hace lo bueno será recompensado por el Señor, sea siervo o libre.  
 6:9 	Amos, traten a sus sierv

 1:16 	Por esta razón se me mostró misericordia, pues ya que soy el peor pecador, Jesucristo pudo mostrar su paciencia infinita como ejemplo para aquellos que eligen creer en él y obtener vida eterna.  
 1:17 	La honra y la gloria sean por siempre y para siempre para el Rey eterno, el inmortal, invisible y único Dios. Amén.  
 1:18 	Estas son las instrucciones que quiero darte, Timoteo, mi hijo, siguiendo las profecías que te han traído hasta aquí, para que puedas pelear la buena batalla.  
 1:19 	Mantén tu fe en Dios y asegúrate de tener una conciencia limpia. Algunos no han querido hacerlo y por eso su fe en Dios ha naufragado.  
 1:20 	Himeneo y Alexander son ejemplo de ello, y yo los he “entregado a Satanás” para que aprendan a no dar una mala imagen de Dios.  
 2:1 	En primer lugar quiero animarte a orar por todos: haz peticiones y agradece en nombre de ellos.  
 2:2 	De este mismo modo ora por los reyes y por todo tipo de líderes, para que podamos tener una vida tranquila y pacíf

 8:13 	Al decir “pacto de una nueva relación,” Dios abandona el primer pacto. Ese pacto que ya está obsoleto y desgastado, y que casi ha desaparecido.  
 9:1 	El antiguo sistema tenía instrucciones sobre cómo adorar, y un santuario terrenal.  
 9:2 	En la primera sala del tabernáculo estaba el candelabro, la mesa, y el pan sagrado. A este lugar se le llamaba el Lugar Santo.  
 9:3 	Al pasar el segundo velo, se encontraba la sala que se llamaba el Lugar Santísimo.  
 9:4 	Dentro de este lugar estaba el altar de oro del incienso, y el “arca del pacto,” cubierta de oro.  Dentro del arca se encontraba una taza de oro que contenía maná, la vara de Aarón que reverdeció, y las inscripciones del pacto sobre piedras. 
 9:5 	Y encima del arca estaba el ángel querubín protegiendo el lugar de la reconciliación. Pero ahora no podemos hablar de esto en detalle.  
 9:6 	Cuando todo esto estuvo establecido, los sacerdotes ya podrían entrar con regularidad a la primera sala para llevar a cabo sus labor

 2:29 	Si ustedes saben que él es bueno y justo, entonces también deben saber que todo el que hace lo justo ha nacido de Dios.  
 3:1 	¡Miren el amor que tiene el Padre para con nosotros! Por eso podemos ser llamados hijos de Dios, ¡porque eso es lo que somos! La razón por la que el mundo no nos reconoce como hijos de Dios es porque no lo reconocen a él.  
 3:2 	Amigos míos, ya somos hijos de Dios, pero lo que llegaremos a ser no se ha revelado todavía. Pero sabemos que cuando él aparezca seremos como él, porque lo veremos como él es realmente.  
 3:3 	Todos los que tienen esta esperanza en él, asegúrense de ser puros, como él lo es.  
 3:4 	Todos los que pecan son violadores de la ley de Dios.  
 3:5 	Pero desde luego ustedes saben que Jesús vino para eliminar los pecados, y en él no hay pecado.  
 3:6 	Todos los que viven en él, ya no pecan más; todos los que siguen pecando es porque no lo han visto y no lo han conocido.  
 3:7 	Queridos amigos, no dejen que nadie los engañe: los que

 7:17 	porque el Cordero que está en medio del trono será su pastor, y los guiará a fuentes de agua viva, y Dios enjugará toda lágrima de sus ojos.  
 8:1 	Cuando abrió el séptimo sello, hubo silencio en el cielo como por media hora.  
 8:2 	Y vi a los siete ángeles que están delante de Dios. A ellos se les dieron siete trompetas.  
 8:3 	Entonces vino otro ángel y se puso en pie en el altar. Tenía un incensario de oro y se le dio una gran cantidad de incienso para añadirlo a las oraciones de todos los santos sobre el altar de oro que está frente al trono.  
 8:4 	Y de la mano del ángel, el humo del incienso subía junto a las oraciones de los santos ante Dios.  
 8:5 	Entonces el ángel tomó el incensario y lo llenó con fuego del altar, y lo lanzó sobre la tierra; y se oyó el sonido de un trueno, con rayos y relámpagos, y un gran terremoto.  
 8:6 	Entonces los siete ángeles que sostenían las siete trompetas se prepararon para hacerlas sonar.  
 8:7 	Y el primer ángel hizo sonar su trom